Part 1: The task involves solving a k-armed bandit problem with 10 arms, each having normally distributed rewards with unknown means. The goal is to learn the expected rewards for each arm using four different methods: (1) a greedy algorithm with non-optimistic initial values, (2) an epsilon-greedy algorithm with varying epsilon values, (3) a greedy algorithm with optimistic initial values, and (4) a gradient bandit algorithm with different learning rates. For each method, the performance is evaluated over 1000 bandit problems by tracking the average reward and the percentage of optimal actions taken at each time step.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constants
k = 10  
n_problems = 1000  
n_steps = 1000  
epsilons = [0.1, 0.01, 0.0] 
alpha = 0.1  
optimistic_initial_value = 5.0  

# Function to generate bandit problems
def generate_bandit_problems(k, n_problems):
    true_means = np.random.normal(0, 1, (n_problems, k))
    return true_means

'''
    This function is the main function to run the bandit problem. It defines a k-armed bandit problem with true_means as the true mean values of the k arms.
    We initially use the environment value that's set up above.
    The main idea is to pull multiple arms and get rewards from them. We then update the q_estimates for each arm based on the rewards we get. 
    The method used here is optimistic, gradient, greedy and epsilon-greedy. 
'''
def run_bandit(true_means, n_steps, method, epsilon=0.1, alpha=0.1, optimistic_initial_value=5.0):
    k = len(true_means)
    q_estimates = np.zeros(k)
    action_counts = np.zeros(k)
    total_reward = 0
    optimal_action_counts = 0
    rewards = np.zeros(n_steps)
    optimal_action_taken = np.zeros(n_steps)
    optimal_action = np.argmax(true_means)
    
    if method == 'optimistic':
        q_estimates.fill(optimistic_initial_value) # Assume every arm has a high value initially
    if method == 'gradient':
        preferences = np.zeros(k)
        action_probabilities = np.ones(k) / k # Set preference based on the reward then update
    
    for step in range(n_steps):
        if method == 'greedy': # Always get the biggest reward
            action = np.argmax(q_estimates)
        elif method == 'epsilon-greedy': # Be greedy or explore with predefined epsilon
            if np.random.rand() < epsilon:
                action = np.random.choice(k)
            else:
                action = np.argmax(q_estimates)
        elif method == 'optimistic':
            action = np.argmax(q_estimates)
        elif method == 'gradient':
            action = np.random.choice(k, p=action_probabilities)
        
        reward = np.random.normal(true_means[action], 1)
        total_reward += reward
        action_counts[action] += 1
        rewards[step] = reward
        if action == optimal_action:
            optimal_action_counts += 1
        optimal_action_taken[step] = optimal_action_counts / (step + 1)
        
        if method == 'gradient':
            one_hot = np.zeros(k)
            one_hot[action] = 1
            baseline = total_reward / (step + 1)
            preferences += alpha * (reward - baseline) * (one_hot - action_probabilities)
            action_probabilities = np.exp(preferences) / np.sum(np.exp(preferences))
        else:
            q_estimates[action] += (reward - q_estimates[action]) / action_counts[action]
    
    return rewards, optimal_action_taken

# Main simulation
true_means = generate_bandit_problems(k, n_problems)
methods = ['greedy', 'optimistic', 'gradient']
results = {method: {'rewards': np.zeros((n_problems, n_steps)), 'optimal_action': np.zeros((n_problems, n_steps))} for method in methods}

# Initialize results for epsilon-greedy with different epsilons
for epsilon in epsilons:
    results[f'epsilon-greedy-{epsilon}'] = {'rewards': np.zeros((n_problems, n_steps)), 'optimal_action': np.zeros((n_problems, n_steps))}

# Loop through all problems and methods
for problem in range(n_problems):
    for method in methods:
        rewards, optimal_action_taken = run_bandit(true_means[problem], n_steps, method, alpha=alpha, optimistic_initial_value=5)
        results[method]['rewards'][problem] = rewards
        results[method]['optimal_action'][problem] = optimal_action_taken
    for epsilon in epsilons:
        rewards, optimal_action_taken = run_bandit(true_means[problem], n_steps, 'epsilon-greedy', epsilon=epsilon)
        results[f'epsilon-greedy-{epsilon}']['rewards'][problem] = rewards
        results[f'epsilon-greedy-{epsilon}']['optimal_action'][problem] = optimal_action_taken

# Averaging results
average_rewards = {method: np.mean(results[method]['rewards'], axis=0) for method in results}
optimal_action_percentages = {method: np.mean(results[method]['optimal_action'], axis=0) for method in results}

# Plotting results
plt.figure(figsize=(12, 8))
for method in average_rewards:
    plt.plot(average_rewards[method], label=f'Average Reward - {method}')
plt.xlabel('Steps')
plt.ylabel('Average Reward')
plt.legend()
plt.title('Average Reward per Step')
plt.show()

plt.figure(figsize=(12, 8))
for method in optimal_action_percentages:
    plt.plot(optimal_action_percentages[method], label=f'Optimal Action % - {method}')
plt.xlabel('Steps')
plt.ylabel('Optimal Action %')
plt.legend()
plt.title('Optimal Action Percentage per Step')
plt.show()


Part 2: The task is to address non-stationary modifications of the k-armed bandit problem, where reward distributions change over time either gradually or abruptly. For gradual changes, two scenarios are considered: (1) a drift change where the mean parameters follow "u*t = u*{t-1} + epsilon*t" with "epsilon_t ~ N(0, 0.001^2)", and (2) a mean-reverting change where "u_t = k \* u*{t-1} + epsilon_t" with "k = 0.5" and "epsilon_t ~ N(0, 0.01^2)". For abrupt changes, at each time step, with a probability of 0.005, the means of the reward distributions are permuted. The evaluation involves comparing three methods: (1) an optimistic greedy approach, (2) epsilon-greedy with a fixed step size, and (3) epsilon-greedy with a decreasing step size. These methods are tested over 1000 repetitions of the non-stationary problems, and the distribution of average rewards at a terminal step (e.g., after 10,000 or 20,000 steps) is analyzed using box plots. The algorithm that yields the most favorable reward distribution at the terminal step is considered preferable. Pilot runs are recommended to fine-tune parameters before the main experiments.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set parameters
k = 10
n_problems = 1000
n_steps = 20000
epsilon_fixed = 0.1
epsilon_decreasing_initial = 0.1
alpha = 0.1 
optimistic_initial_value = 5.0


def initialize_bandit(k):
    means = np.random.normal(0, 1, k)
    return means

#Functions for non-stationary bandit problems
# Function to apply drift change
def apply_drift_change(means):
    means += np.random.normal(0, 0.001, k)
    return means

# Function to apply mean-reverting change
def apply_mean_reverting_change(means):
    means = 0.5 * means + np.random.normal(0, 0.01, k)
    return means

# Function to apply abrupt change
def apply_abrupt_change(means):
    if np.random.rand() < 0.005:
        np.random.shuffle(means)
    return means

# Main function to run the bandit problem
def run_bandit(method, k, n_steps, initial_means, epsilon=None, alpha=None, optimistic_value=None):
    q_estimates = np.zeros(k)
    if method == 'optimistic':
        q_estimates.fill(optimistic_value)
    action_counts = np.zeros(k)
    rewards = np.zeros(n_steps)
    optimal_action_counts = np.zeros(n_steps)

    means = initial_means.copy()
    optimal_action = np.argmax(means)

    for t in range(n_steps):
        if method == 'epsilon_fixed':
            if np.random.rand() < epsilon:
                action = np.random.randint(k)
            else:
                action = np.argmax(q_estimates)
        elif method == 'epsilon_decreasing':
            epsilon_t = epsilon_decreasing_initial / (t + 1)
            if np.random.rand() < epsilon_t:
                action = np.random.randint(k)
            else:
                action = np.argmax(q_estimates)
        elif method == 'optimistic':
            action = np.argmax(q_estimates)

        reward = np.random.normal(means[action], 1)
        action_counts[action] += 1
        rewards[t] = reward
        optimal_action_counts[t] = (action == optimal_action)

        if method == 'epsilon_fixed':
            q_estimates[action] += alpha * (reward - q_estimates[action])
        else:
            q_estimates[action] += (reward - q_estimates[action]) / action_counts[action]

        means = apply_drift_change(means)  
        means = apply_abrupt_change(means)  
        optimal_action = np.argmax(means)

    return rewards, optimal_action_counts

# Running experiments
methods = ['optimistic', 'epsilon_fixed', 'epsilon_decreasing']
results = {method: [] for method in methods}

# Loop through all methods and problems
for method in methods:
    for _ in range(n_problems):
        initial_means = initialize_bandit(k)
        if method == 'optimistic':
            rewards, optimal_action_counts = run_bandit(method, k, n_steps, initial_means, optimistic_value=optimistic_initial_value)
        elif method == 'epsilon_fixed':
            rewards, optimal_action_counts = run_bandit(method, k, n_steps, initial_means, epsilon=epsilon_fixed, alpha=alpha)
        elif method == 'epsilon_decreasing':
            rewards, optimal_action_counts = run_bandit(method, k, n_steps, initial_means, epsilon=epsilon_decreasing_initial)
        results[method].append(rewards)

# Plotting the results
for method in methods:
    final_rewards = [result[-1] for result in results[method]]
    plt.boxplot(final_rewards, positions=[methods.index(method)], widths=0.6)

plt.xticks(range(len(methods)), methods)
plt.ylabel('Final Reward Distribution')
plt.title('Comparison of Methods on Non-Stationary Bandit Problem')
plt.show()
